**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


In the Intro to SQL micro-course, you learned how to use aggregate functions, which perform calculations based on sets of rows. In this tutorial, you'll learn how to define **analytic functions**, which also operate on a set of rows. However, unlike aggregate functions, analytic functions return a (potentially different) value for each row in the original table.

Analytic functions allow us to perform complex calculations with relatively straightforward syntax. For instance, we can quickly calculate moving averages and running totals, among other quantities.

### Syntax
To understand how to write analytic functions, we'll work with a small table containing data from two different people who are training for a race. The id column identifies each runner, the date column holds the day of the training session, and time shows the time (in minutes) that the runner dedicated to training. Say we'd like to calculate a moving average of the training times for each runner, where we always take the average of the current and previous training sessions. We can do this with the following query:

![](https://i.imgur.com/rehp8HM.png)

All analytic functions have an **OVER** clause, which defines the sets of rows used in each calculation. The **OVER** clause has three (optional) parts:

- The **PARTITION BY** clause divides the rows of the table into different groups. In the query above, we divide by id so that the calculations are separated by runner.
- The **ORDER BY** clause defines an ordering within each partition. In the sample query, ordering by the date column ensures that earlier training sessions appear first.
- The final clause (**ROWS BETWEEN 1 PRECEDING AND CURRENT ROW**) is known as a **window frame** clause. It identifies the set of rows used in each calculation. We can refer to this group of rows as a window. (Actually, analytic functions are sometimes referred to as **analytic window functions** or simply **window functions**!)

![](https://i.imgur.com/GjiKlA7.png)

### (More on) window frame clauses
There are many ways to write window frame clauses:

- **ROWS BETWEEN 1 PRECEDING AND CURRENT ROW** - the previous row and the current row.
- **ROWS BETWEEN 3 PRECEDING AND 1 FOLLOWING** - the 3 previous rows, the current row, and the following row.
- **ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING** - all rows in the partition.

Of course, this is not an exhaustive list, and you can imagine that there are many more options! In the code below, you'll see some of these clauses in action.

### Three types of analytic functions
The example above uses only one of many analytic functions. BigQuery supports a wide variety of analytic functions, and we'll explore a few here. For a complete listing, you can take a look at the documentation.

1) Analytic aggregate functions
As you might recall, **AVG()** (from the example above) is an aggregate function. The **OVER** clause is what ensures that it's treated as an analytic (aggregate) function. **Aggregate functions** take all of the values within the window as input and return a single value.

- **MIN()** (or **MAX()**) - Returns the minimum (or maximum) of input values
- **AVG()** (or **SUM()**) - Returns the average (or sum) of input values
- **COUNT()** - Returns the number of rows in the input

2) Analytic navigation functions
**Navigation functions** assign a value based on the value in a (usually) different row than the current row.

- **FIRST_VALUE()** (or **LAST_VALUE()**) - Returns the first (or last) value in the input
- **LEAD()** (and **LAG()**) - Returns the value on a subsequent (or preceding) row

3) Analytic numbering functions
Numbering functions assign integer values to each row based on the ordering.

- **ROW_NUMBER()** - Returns the order in which rows appear in the input (starting with 1)
- **RANK()** - All rows with the same value in the ordering column receive the same rank value, where the next row receives a rank value which increments by the number of rows with the previous rank value.

# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,a93783ff83730b096673d6496e0a73817616b1e0,7602478d4ed02f3137d6f55c94cc89d74735b5482ec7d0...,2018-03-31 21:30:00+00:00,2018-03-31 21:30:00+00:00,622,3.7,NaN,NaN,NaN,NaN,...,0.0,10.00,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
1,106194b2c504449060b90988e8cfa7f1c9cadc21,7602478d4ed02f3137d6f55c94cc89d74735b5482ec7d0...,2018-04-01 00:30:00+00:00,2018-04-01 00:30:00+00:00,822,4.9,NaN,NaN,NaN,NaN,...,0.0,12.60,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
2,78a6796eefbfb9bd9c8df0471285cbdda0a5fe83,7602478d4ed02f3137d6f55c94cc89d74735b5482ec7d0...,2018-04-01 02:45:00+00:00,2018-04-01 03:00:00+00:00,935,9.5,NaN,NaN,NaN,NaN,...,0.0,25.00,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
3,9a9bb9626dd403973ec6e5763a95d22a417f2425,7602478d4ed02f3137d6f55c94cc89d74735b5482ec7d0...,2018-04-01 12:45:00+00:00,2018-04-01 12:45:00+00:00,10,0.1,NaN,NaN,NaN,NaN,...,0.0,2.20,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
4,8ae6bf9ce9aae43900a468259c611c8c22dfd9a9,ddbfdb4b546c3b981dee3345edaed1bd5ad14557d682cf...,2018-03-25 06:30:00+00:00,2018-03-25 06:45:00+00:00,302,3.3,NaN,NaN,NaN,NaN,...,0.0,0.01,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to December 31, 2017.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 15 days and the following 15 days, as long as the days fit within the two-year time frame.  For instance, when calculating the value in this column for January 5, 2016, the window will include the number of trips for the preceding 4 days, the current date, and the following 15 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) micro-course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2018-01-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,trip_date,avg_num_trips
0,2016-01-08,81694.043478
1,2016-05-13,95071.000000
2,2016-01-19,81461.870968
3,2016-11-17,81056.806452
4,2016-12-03,80221.258065


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# # Lines below will give you a hint or solution code
# q_1.hint()
# q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() 
                            OVER(
                                PARTITION BY pickup_community_area
                                ORDER BY trip_start_timestamp
                                ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2017-05-01'
                    """

# Check your answer
q_2.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,39.0,2017-05-01 01:15:00+00:00,2017-05-01 01:30:00+00:00,1
1,39.0,2017-05-01 02:15:00+00:00,2017-05-01 02:45:00+00:00,2
2,39.0,2017-05-01 05:15:00+00:00,2017-05-01 05:30:00+00:00,3
3,39.0,2017-05-01 05:45:00+00:00,2017-05-01 06:15:00+00:00,4
4,39.0,2017-05-01 06:45:00+00:00,2017-05-01 07:00:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
# q_2.hint()
# q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://i.imgur.com/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2017-05-01' 
                   """
# Check your answer
q_3.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,1b7aef0344f45914e7942efad05553934fe222c880a1cb...,2017-05-01 04:45:00+00:00,2017-05-01 05:00:00+00:00,NaN
1,1be1407efca4ff6ac787c8794df57e8eeac031eaf08fca...,2017-05-01 01:15:00+00:00,2017-05-01 01:15:00+00:00,NaN
2,1e5b69c83640bd5225fb3ccc33bc989b859f3e4cdb59f6...,2017-05-01 23:45:00+00:00,2017-05-02 00:00:00+00:00,150.0
3,24380d3a358aef97432d7b362fe55415819da221fcd800...,2017-05-01 00:00:00+00:00,2017-05-01 00:30:00+00:00,NaN
4,24380d3a358aef97432d7b362fe55415819da221fcd800...,2017-05-01 00:45:00+00:00,2017-05-01 00:45:00+00:00,15.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
# q_3.hint()
# q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*